## Analyze NCP scans for instrumental polarization and ionospheric Faraday rotation
### A. Ordog, July 2022
### NOTE: please do not modify

## Import packages and define file directory

In [ ]:
import dva_sdhdf_combine_simple
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from astropy.modeling import models, fitting
import ephem
from astropy.coordinates import SkyCoord,EarthLocation, AltAz, ICRS, Galactic, FK4, FK5
from scipy import interpolate

#### Change the directory to where the files are located" ####
dir_in = '/home2/DATA_AO/DVA_DATA/'
dir_out = '/media/ordoga/DVA_data/'
##############################################################

## Read in NCP data files and stitch them together (averaged into freq. bins)
### ***Note: skip this if files already made

In [ ]:
%%time

imp.reload(dva_sdhdf_combine_simple)

#NCPdays = ['Feb18_NCP_part1']
NCPdays = ['Feb18_NCP_part1','Feb18_NCP_part2','Feb18_NCP_part3','Feb18_NCP_part4']
dates = ['2022-02-18','2022-02-19','2022-02-20','2022-02-21']

outname = 'NCP_Feb18'
dva_sdhdf_combine_simple.combine(dir_in,dir_out,NCPdays,outname,freq_s=75,freq_avg=True)

## Read in stitched together, frequency averaged NCP data:

In [ ]:
file = h5py.File('/media/ordoga/DVA_data/NCP_Feb18.h5','r+')

RR_set = file['data']['beam_0']['band_SB0']['scan_0']['data'][:,0,:] 
LL_set = file['data']['beam_0']['band_SB0']['scan_0']['data'][:,1,:] 
reRL_set = file['data']['beam_0']['band_SB0']['scan_0']['data'][:,2,:]
imRL_set = file['data']['beam_0']['band_SB0']['scan_0']['data'][:,3,:]
t_set = file['data']['beam_0']['band_SB0']['scan_0']['metadata']['utc']
az_set = file['data']['beam_0']['band_SB0']['scan_0']['metadata']['azimuth']
el_set = file['data']['beam_0']['band_SB0']['scan_0']['metadata']['elevation']
freq = file['data']['beam_0']['band_SB0']['frequency'][:]/1e6

t_set_plt = Time(t_set, format='isot',scale='utc').mjd

df = freq[1]-freq[0]

print('Frequency spacing: ',df)
print('Shape of time array: ',t_set_plt.shape)
print('Shape of data array: ',reRL_set.shape)

### Sanity check to make sure averaged data yields correct spectrum

In [ ]:
ii = 70
print(freq[ii])
plt.scatter(freq[:],10*np.log10(LL_set[50,:]),s=5)
plt.scatter(freq[ii],10*np.log10(LL_set[50,ii]),s=15,color='red')

## Read in temperature data
### Note: only used to observe trends - not used in calcualtions

In [ ]:
def month_to_num(month_name):
    if month_name == 'Jan': month_num = '01'
    if month_name == 'Feb': month_num = '02'
    if month_name == 'Mar': month_num = '03'
    if month_name == 'Apr': month_num = '04'
    if month_name == 'May': month_num = '05'
    if month_name == 'Jun': month_num = '06'
    if month_name == 'Jul': month_num = '07'
    if month_name == 'Aug': month_num = '08'
    if month_name == 'Sep': month_num = '09'
    if month_name == 'Oct': month_num = '10'
    if month_name == 'Nov': month_num = '11'
    if month_name == 'Dec': month_num = '12'
    return(month_num)

i = 0
t_weath = []
temp_C = []
   
with open("../DVA2/DATA/weather/weather_Feb18_NCP.txt") as fp:
    for line in fp:
        t_weath.append(str( line.split()[2]+'-'+month_to_num(line.split()[1])+'-'+line.split()[0]+
                  'T'+line.split()[3]))
        temp_C.append(line.split()[4])
with open("../DVA2/DATA/weather/weather_Feb19_NCP.txt") as fp:
    for line in fp:
        t_weath.append(str( line.split()[2]+'-'+month_to_num(line.split()[1])+'-'+line.split()[0]+
                  'T'+line.split()[3]))
        temp_C.append(line.split()[4])
with open("../DVA2/DATA/weather/weather_Feb20_NCP.txt") as fp:
    for line in fp:
        t_weath.append(str( line.split()[2]+'-'+month_to_num(line.split()[1])+'-'+line.split()[0]+
                  'T'+line.split()[3]))
        temp_C.append(line.split()[4])
with open("../DVA2/DATA/weather/weather_Feb21_NCP.txt") as fp:
    for line in fp:
        t_weath.append(str( line.split()[2]+'-'+month_to_num(line.split()[1])+'-'+line.split()[0]+
                  'T'+line.split()[3]))
        temp_C.append(line.split()[4])

temp_C = np.array(temp_C,dtype=float)
t_weath_fix = Time(t_weath, format='isot',scale='utc')
t_weath_plt = t_weath_fix.mjd

## Calculate sunset and sunrise times

In [ ]:
obs = ephem.Observer()
obs.lon  = str(-119.6) #Note that lon should be in string format
obs.lat  = str(49.3)      #Note that lat should be in string format
obs.elev = 546
obs.date = "2022-02-18 00:00:00.0"
loc = EarthLocation(lat = 49.3*u.deg, lon = -119.6*u.deg, height = 546*u.m)

tsunset = []
tsunrise = []
tsunset_plt = []
tsunrise_plt = []
for i in range(0,4):    
    tsunset.append(obs.next_setting(ephem.Sun()).datetime())
    tsunrise.append(obs.next_rising(ephem.Sun()).datetime()) 
    print(tsunset[i],tsunrise[i])
    tsunset_plt.append(Time(str(tsunset[i]),format='iso',scale='utc').mjd)
    tsunrise_plt.append(Time(str(tsunrise[i]),format='iso',scale='utc').mjd)
    obs.date = tsunrise[i]

## Plot reRL and imRL along with temperature vs time at chosen frequency

In [ ]:
######################################
fr_plot = 800
t1 = '2022-02-18T00:00:00.0Z'
t2 = '2022-02-21T23:00:00.0Z'
######################################

t1_plt = Time(t1,format='isot',scale='utc').mjd
t2_plt = Time(t2,format='isot',scale='utc').mjd

fs = 18
fig,axs = plt.subplots(2,1,figsize=(16,8))    

w = np.where(abs(freq-fr_plot)<df)[0][0]
print(freq[w])

axs[0].scatter(t_set_plt,reRL_set[:,w], color='C0',s=0.2,label='reRL')
axs[0].set_ylim(-8e5,-5e5)
axs[1].scatter(t_set_plt,imRL_set[:,w], color='C1',s=0.2,label='imRL')
axs[1].set_ylim(-1.5e5,1.5e5)

for i in range(0,2):
    axs[i].xaxis.set_major_locator(HourLocator(interval=12))
    axs[i].set_ylabel('Power (raw spectrometer units)')
    axs[i].legend(loc=1,markerscale=10)
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M')
    axs[i].set_xlabel('Time (UTC)')
    axs[i].grid()
    axs[i].set_xlim(t1_plt,t2_plt)    
    ax1 = axs[i].twinx()
    ax1.plot(t_weath_plt,temp_C,color='black')
    ax1.set_ylim(-10,10)
    ax1.set_ylabel('Outside Temperature (C)')
    for j in range(0,4):
        axs[i].axvspan(tsunset_plt[j],tsunrise_plt[j],alpha=0.2,facecolor='gray',
                       zorder=0,linewidth=1,edgecolor='black')
        

## Read in the noise source data

In [ ]:
t_noise1 = []
t_noise2 = []

with open("../DVA2/DATA/noise_times/noise_Feb18_NCP.txt") as fp:
    for line in fp:       
        t_noise1.append(line.split()[0]+'T'+line.split()[1][0:12])
        t_noise2.append(line.split()[2]+'T'+line.split()[3][0:12])
with open("../DVA2/DATA/noise_times/noise_Feb19_NCP.txt") as fp:
    for line in fp:       
        t_noise1.append(line.split()[0]+'T'+line.split()[1][0:12])
        t_noise2.append(line.split()[2]+'T'+line.split()[3][0:12])
with open("../DVA2/DATA/noise_times/noise_Feb20_NCP.txt") as fp:
    for line in fp:       
        t_noise1.append(line.split()[0]+'T'+line.split()[1][0:12])
        t_noise2.append(line.split()[2]+'T'+line.split()[3][0:12])

t_noise1_fix = Time(t_noise1, format='isot',scale='utc')
t_noise2_fix = Time(t_noise2, format='isot',scale='utc')

t_noise1_plt = t_noise1_fix.mjd
t_noise2_plt = t_noise2_fix.mjd

## Calculate on and off noise power

In [ ]:
t_buff = 5./(3600.*24.) # seconds
t_off = 30./(3600.*24.) # seconds

wnoise = []
wnoise_rem = []
woff = []

pnoiseLL = np.empty([len(t_noise1),len(freq)])
poffLL = np.empty([len(t_noise1),len(freq)])
pnoiseRR = np.empty([len(t_noise1),len(freq)])
poffRR = np.empty([len(t_noise1),len(freq)])
tnoise = np.empty([len(t_noise1)])

for i in range(0,len(t_noise1)):
    wnoise_sub = np.where( (t_set_plt >= t_noise1_plt[i]+t_buff) & (t_set_plt <= t_noise2_plt[i]-t_buff) )[0]
    wnoise_sub_rem = np.where( (t_set_plt >= t_noise1_plt[i]-t_buff) & (t_set_plt <= t_noise2_plt[i]+t_buff) )[0]
    
    woff1 = np.where( (t_set_plt >= t_noise1_plt[i]-t_buff-t_off) & 
                      (t_set_plt <= t_noise1_plt[i]-t_buff) )[0]
    woff2 = np.where( (t_set_plt >= t_noise2_plt[i]+t_buff) & 
                      (t_set_plt <= t_noise2_plt[i]+t_buff+t_off) )[0]

    wnoise=wnoise+list(wnoise_sub)
    wnoise_rem=wnoise_rem+list(wnoise_sub_rem)
    woff = woff+list(woff1)+list(woff2)
    
    pnoiseLL[i,:] = np.nanmean(LL_set[wnoise_sub,:],axis=0)
    pnoiseRR[i,:] = np.nanmean(RR_set[wnoise_sub,:],axis=0)
    poffLL[i,:] = np.nanmean(LL_set[list(woff1)+list(woff2),:],axis=0)
    poffRR[i,:] = np.nanmean(RR_set[list(woff1)+list(woff2),:],axis=0)
    tnoise[i] = np.nanmean(t_set_plt[wnoise_sub])

wall = list(np.array(np.linspace(0,len(t_set_plt)-1,len(t_set_plt)),dtype=int))
print('Total data points:',len(wall),len(t_set_plt))
wkeep = [x for x in wall if x not in wnoise_rem]
print('Data points without noise:',len(wkeep))

## Check the noise power on and off (zoomed in in time)

In [ ]:
######################################
fr_plot_arr = 800
I_scl1 = 71
I_scl2 = 78
######################################

fs = 18
fig,axs = plt.subplots(4,1,figsize=(16,15))    

w = np.where(abs(freq-fr_plot_arr)<df)[0][0]
print(freq[w])

for i in range(0,4):

    t1_plt = Time(dates[i]+'T00:00:00.0Z',format='isot',scale='utc').mjd
    t2_plt = Time(dates[i]+'T06:00:00.0Z',format='isot',scale='utc').mjd

    axs[i].plot(t_set_plt,10*np.log10(RR_set[:,w]),label='RR',color='C0',zorder=0)
    axs[i].plot(t_set_plt,10*np.log10(LL_set[:,w]),label='LL',color='C1',zorder=0)
    axs[i].scatter(t_set_plt[wkeep],10*np.log10(RR_set[wkeep,w]),label='RR',color='blue',zorder=1,s=0.2)
    axs[i].scatter(t_set_plt[wkeep],10*np.log10(LL_set[wkeep,w]),label='LL',color='red',zorder=1,s=0.2)
    axs[i].scatter(tnoise,10*np.log10(pnoiseRR[:,w]),color='black',s=10,zorder=10)
    axs[i].scatter(tnoise,10*np.log10(pnoiseLL[:,w]),color='black',s=10,zorder=10)
    axs[i].scatter(tnoise,10*np.log10(poffRR[:,w]),color='black',s=10,zorder=10)
    axs[i].scatter(tnoise,10*np.log10(poffLL[:,w]),color='black',s=10,zorder=10)
    axs[i].set_ylim(I_scl1,I_scl2)
    axs[i].set_ylabel('Power (dB)')

    axs[i].legend(loc=1,markerscale=10)
    axs[i].xaxis.set_major_locator(MinuteLocator(interval=30))
    #axs[i].set_xticks(loc=HourLocator(interval=2))
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)')
    axs[i].grid()
    axs[i].set_xlim(t1_plt,t2_plt)    
    if i == 0:
        axs[i].set_title('NCP observations at 800 MHz (freq. channels binned to '+str(df)+' MHz)')


## Calculate the gains

In [ ]:
RR_med_noise = np.nanmedian(pnoiseRR-poffRR,axis=0)
LL_med_noise = np.nanmedian(pnoiseLL-poffLL,axis=0)

G_RR = np.empty_like(RR_set)
G_LL = np.empty_like(LL_set)

for i in range(0,len(freq)):
    
    f_RR_interp = interpolate.interp1d(tnoise, pnoiseRR[:,i]-poffRR[:,i],fill_value="extrapolate",kind='linear')
    G_RR[:,i] = f_RR_interp(t_set_plt)/RR_med_noise[i]
    
    f_LL_interp = interpolate.interp1d(tnoise, pnoiseLL[:,i]-poffLL[:,i],fill_value="extrapolate",kind='linear')
    G_LL[:,i] = f_LL_interp(t_set_plt)/LL_med_noise[i]

## Apply gain corrections to RR and LL and visualize the result

In [ ]:
######################################
fr_plot = 800
I_scl1 = 71
I_scl2 = 74
######################################

fs = 18
fig,axs = plt.subplots(4,1,figsize=(16,15))    

w = np.where(abs(freq-fr_plot)<df)[0][0]
print(freq[w])

for i in range(0,4):

    t1_plt = Time(dates[i]+'T00:00:00.0Z',format='isot',scale='utc').mjd
    t2_plt = Time(dates[i]+'T23:59:59.9Z',format='isot',scale='utc').mjd

    axs[i].scatter(t_set_plt,10*np.log10(RR_set[:,w]),label='RR',color='C0',s=0.1)
    axs[i].scatter(t_set_plt,10*np.log10(LL_set[:,w]),label='LL',color='C1',s=0.1)
    axs[i].scatter(t_set_plt,10*np.log10(RR_set[:,w]/G_RR[:,w]),label='RR',color='blue',s=0.1)
    axs[i].scatter(t_set_plt,10*np.log10(LL_set[:,w]/G_LL[:,w]),label='LL',color='red',s=0.1)
    axs[i].set_ylim(I_scl1,I_scl2)
    axs[i].set_ylabel('Power (dB)')
    
    ax1 = axs[i].twinx()
    ax1.scatter(t_set_plt,G_RR[:,w],color='gray',s=0.1,label='G_RR')
    ax1.scatter(t_set_plt,G_LL[:,w],color='black',s=0.1,label='G_LL')
    #ax1.scatter(tnoise,(pnoiseLL[:,w]-poffLL[:,w])/LL_med_noise[w],color='black',s=10)
    ax1.set_ylim(0.85,1.15)

    axs[i].legend(loc=1,markerscale=10)
    ax1.legend(loc=2,markerscale=10)
    axs[i].xaxis.set_major_locator(HourLocator(interval=2))
    #axs[i].set_xticks(loc=HourLocator(interval=2))
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].set_xlabel('Time (UTC)')
    axs[i].grid()
    axs[i].set_xlim(t1_plt,t2_plt)    
    if i == 0:
        axs[i].set_title('NCP observations at 800 MHz (freq. channels binned to '+str(df)+' MHz)')